<a href="https://colab.research.google.com/github/lili-alsh/Payment/blob/master/%D0%90%D0%B2%D1%82%D0%BE%D0%B7%D0%B0%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_xlsx_%D1%81_%D1%83%D1%81%D0%BB%D0%BE%D0%B2%D0%B8%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string


# Путь для файла со всеми платежами
path_all_pmnt = "/content/drive/My Drive/Обновление реестра платежей/Выгрузка.xlsx"
pmnts = pd.read_excel(path_all_pmnt)
# Список контрагентов, не включаемых в реестр
cntr_except = []
pmnts = pmnts[~pmnts['Контрагент'].isin(cntr_except)]
# Список исключений из контрагентов. Можно расширять, если при выполнении кода по проверке контрагентов к исключению
# вышли в список контрагенты, которых нужно оставить
individ_grant = []
pmnts['Контрагент'] = pmnts['Контрагент'].astype('string')
# Создаем список из латинских прописных букв для отбора иностранных контрагентов
x = string.ascii_uppercase
list = [ch for ch in x]
# Список исключений по иностранным контрагентам-физическим лицам
list_foreign=[]

# Нужно запустить вначале с этим кодом, чтобы проверить, какие контрагенты исключаются
pmnts = pmnts[(pmnts['Контрагент'].str.contains('ООО')==False) & 
              (pmnts['Контрагент'].str.contains('АО')==False) &
              (pmnts['Контрагент'].str.contains('"')==False)&
              (pmnts['Контрагент'].str.contains('Самозанят')==False)&
              (pmnts['Контрагент'].str.contains('самозанят')==False)&
              (pmnts['Контрагент'].str.contains('ИП')==False)&
              (~pmnts['Контрагент'].isin(individ_grant))& 
              (~pmnts['Контрагент'].str[0].isin(list)) |
              (pmnts['Контрагент'].isin(list_foreign))]
for i in pmnts['Контрагент']:
  print(i)

In [ ]:
import pandas as pd
import string
from openpyxl import load_workbook
from openpyxl.styles import Font, Color
from openpyxl.utils.dataframe import dataframe_to_rows
import locale
import datetime

# Путь к файлу текущих платежей
path_curr_pmnt = "/content/drive/My Drive/Обновление реестра платежей/Все платежи на.xlsx"
sheet = "платежи (3)" 

# Читаем файл выгрузки с платежами из Битрикса
pmnts = pd.read_excel(path_all_pmnt)
# Убираем контрагента - юридическое лицо резидента РФ
pmnts = pmnts[~pmnts['Контрагент'].isin(cntr_except)]
pmnts = pmnts[pmnts['Описание'] != 'Удалить']
# Список исключений из контрагентов. Можно расширять, если при выполнении кода по проверке контрагентов к исключению
# вышли в список контрагенты, которых нужно оставить
individ_grant = []
pmnts['Контрагент'] = pmnts['Контрагент'].astype('string')
# Создаем список из латинских прописных букв для отбора иностранных контрагентов
x = string.ascii_uppercase
list = [ch for ch in x]
# Список исключений по иностранным консультантам
list_foreign=[]

# Переименовываем столбцы как в файле Все платежи
pmnts = pmnts.rename(columns={'Unnamed: 0':'№', 'Описание':'Сделка','Сумма в рублях':'Сумма к оплате', 'Бюджет':'Название бюджета'})
# Оставляем в списке контрагентов только юр лиц (в т.ч. иностранных), ИП и самозанятых
pmnts = pmnts[(pmnts['Контрагент'].str.contains('ООО')) | 
              (pmnts['Контрагент'].str.contains('АО')) |
              (pmnts['Контрагент'].str.contains('"'))|
              (pmnts['Контрагент'].str.contains('Самозанят'))|
              (pmnts['Контрагент'].str.contains('самозанят'))|
              (pmnts['Контрагент'].str.contains('ИП'))|
              (pmnts['Контрагент'].isin(individ_grant))|
              (pmnts['Контрагент'].str[0].isin(list))|
              (pmnts['Контрагент'] == "")]
# Убираем иностранных консультантов
pmnts = pmnts[(~pmnts['Контрагент'].isin(list_foreign))]
# Убираем из выгруженных заявок тестовые заявки
pmnts = pmnts[(round(pmnts['Сумма к оплате'], 2) != 0.0)&
              (round(pmnts['Сумма к оплате'], 2) != 1.0)&
              (round(pmnts['Сумма к оплате'], 2) != 10.0)&
              (round(pmnts['Сумма к оплате'], 2) != 20.0)&
              (round(pmnts['Сумма к оплате'], 2) != 3.0)&
              (round(pmnts['Сумма к оплате'], 2) != 2.0)&
              (round(pmnts['Сумма к оплате'], 2) != 5.0)&
              (~pmnts['Сделка'].str.contains('Тестов'))]

# Переставить колонку Сумма к оплате
cols = pmnts.columns.tolist()
cols = cols[0:3] + cols[-1:] + cols[3:11]
pmnts = pmnts[cols]

# Читаем файл со всеми платежами на прошлый день (!не забыть убрать оплаченные заявки!)

all_pmnts = pd.read_excel(path_curr_pmnt)
all_pmnts = all_pmnts.rename(columns={'Дата созда':'Дата создания'})

# Читаем файл со всеми платежами            
wb = load_workbook(path_curr_pmnt)
ws = wb[sheet]

del_pmnts = all_pmnts.merge(pmnts, how='outer', left_on='№',right_on='№')
del_pmnts = del_pmnts[(del_pmnts['Статус_y'].isna() == True) &
                      (del_pmnts['Статья бюджета_x'].isna() == False) & 
                      (~del_pmnts['Контрагент_x'].isin(cntr_except))]
del_row = []
if del_pmnts.shape[0] != 0:
  for row in ws.iter_rows(min_row=2, max_col=1):
    for cell in row:
      for idx, r in del_pmnts.iterrows():
        if cell.value == del_pmnts.loc[idx]['№']:
          del_row.append(cell.row)

for ind, rw in enumerate(del_row):
  if ind == 0:
    ws.delete_rows(rw)
  else:
    ws.delete_rows(rw-ind)

# Обновляем файл со всеми платежами
all_pmnts = all_pmnts.merge(pmnts, left_on='№',right_on='№')
curr_pmnts = all_pmnts[['№', 'Сделка_x', 'Статус_y','Сумма к оплате_y','Контрагент_y',
           'Название бюджета_y','Номер счета_y', 'Дата создания_y',
           'Автор заявки_y', 'Договор_y', 'Проект_y', 'Статья бюджета_y']]

# Отдельно записываем новые платежи
new_pmnts = all_pmnts.merge(pmnts, how='outer', left_on='№',right_on='№')
new_pmnts = new_pmnts[new_pmnts['Сделка_x'].isna() == True]
new_pmnts = new_pmnts [['№', 'Сделка', 'Статус','Сумма к оплате','Контрагент',
           'Название бюджета','Номер счета', 'Дата создания',
           'Автор заявки', 'Договор', 'Проект', 'Статья бюджета']]
date = pd.to_datetime(new_pmnts['Дата создания'], dayfirst=True).dt.strftime('%d.%m.%Y')
new_pmnts['Дата создания'] = date
new_pmnts['№'] = new_pmnts['№'].astype('int32')

# Находим позицию ячейки в изначальном файле, с которой надо начать обновление
for row in ws.iter_rows(min_row=2, max_col=1, max_row=all_pmnts.shape[0]):
   for cell in row:
     if cell.value == curr_pmnts.iloc[0,0]:
       position = cell.row
       break

# Приводим к формату даты столбец Дата создания
date = pd.to_datetime(curr_pmnts['Дата создания_y'], dayfirst=True)
curr_pmnts['Дата создания_y'] = date

# Вносим изменения в файл согласно обновленным данным
for col in range(curr_pmnts.shape[1]):
  for row in range(curr_pmnts.shape[0]):
    if col == 7:
      value = curr_pmnts.iloc[row, col]
      cell = ws.cell(position+row, col+1)
      cell.value = value
      cell.number_format = 'DD.MM.YYYY'
    else:
      ws.cell(position+row, col+1).value = curr_pmnts.iloc[row, col]    

def ins_row (a_pmnts, numb):
  # Вставляем строки для добавления новых платежей
  ws.insert_rows(numb, a_pmnts.shape[0])
  # Добавляем новые платежи
  for col in range(a_pmnts.shape[1]):
    for row in range(a_pmnts.shape[0]):
      if col == 7:
        value = datetime.datetime.strptime(a_pmnts.iloc[row, col], "%d.%m.%Y")
        cell = ws.cell(numb+row, col+1)
        cell.value = value
        cell.number_format = 'DD.MM.YYYY'
      else:
        ws.cell(numb+row, col+1).value = a_pmnts.iloc[row, col]
        if col == 3:
          ws.cell(numb+row, col+1).number_format = '# ### 000.00'

ins_row(new_pmnts, position)

for r in range(ws.max_row-2):
  ws.row_dimensions[r].height = 15

new_position = new_pmnts.shape[0]

# for row in ws.iter_rows(min_row=2, max_col=1):
#    for cell in row:
#      if cell.value == " Заявка на приобретение Согл-ТМЦиУ-8990":
#        end_row = cell.row


for row in ws.iter_rows(min_row=2, max_col=1, max_row = end_row-1):
  for cell in row:
    cell.hyperlink = None
url = "https://portal.university.innopolis.ru/processes/list/111/element/0/{}/"
for row in ws.iter_rows(min_row=2, max_col=1, max_row = end_row-1):
  for cell in row:
    value = cell.value
    cell.value = '=HYPERLINK("%s", "%s")' % (url.format(value), value)

# ws.cell(end_row+1, 1).value = '=HYPERLINK("http://megaplan.innopolis.edu.ru/deals/17401/card/", " Заявка на приобретение Согл-ТМЦиУ-12627")'
# ws.cell(end_row+2, 1).value = '=HYPERLINK("http://megaplan.innopolis.edu.ru/deals/16351/card/", " Заявка на приобретение Согл-ТМЦиУ-11589")'
# ws.cell(end_row+3, 1).value = '=HYPERLINK("http://megaplan.innopolis.edu.ru/deals/15710/card/", " Заявка на приобретение Согл-ТМЦиУ-11126")'
# ws.cell(end_row+4, 1).value = '=HYPERLINK("http://megaplan.innopolis.edu.ru/deals/8987/card/", " Заявка на приобретение Согл-ТМЦиУ-9853")'

wb.save('Тест.xlsx')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
